In [ ]:
from auth import get_auth

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config(auth = get_auth())

In [ ]:
!ls

In [ ]:
from azureml.core import Experiment
from azureml.core import Run

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

#from azureml.core import Experiment
experiment_name = "My-first-Test-Experiment"
experiment = Experiment(workspace=ws, name=experiment_name)
print(experiment)


cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_D2_v2',
       max_nodes=1)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
compute_target

In [ ]:
from azureml.core import Dataset
dataset = Dataset.get_by_name(ws, name='measure_table')

In [ ]:
import os
import shutil
print("Creating temp folder...")
temp_path = "tmp_train"
if os.path.exists(temp_path):
    shutil.rmtree(temp_path)
os.mkdir(temp_path)

print("Copying files...")
shutil.copy(os.path.join("code", "model_alpha_1.0.pkl"), temp_path)
shutil.copy(os.path.join("code", "evaluation.py"), temp_path)
print("Done.")

In [ ]:
from azureml.train.estimator import Estimator

pip_packages = [
    "azureml-dataprep[fuse,pandas]",
    "glob2",
    "sklearn",
    "joblib"
]

# Create the estimator.
estimator = Estimator(
    source_directory=temp_path,
    compute_target=compute_target,
    entry_script="evaluation.py",
    inputs=[dataset.as_named_input("dataset")],
    pip_packages=pip_packages
)

# Set compute target.
estimator.run_config.target = compute_target

# Run the experiment.
run = experiment.submit(estimator)

# Show outpus.
from azureml.widgets import RunDetails
RunDetails(run).show()

In [ ]:
run

In [ ]:
run.get_details_with_logs()